In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import scipy.io as spi
%matplotlib inline

Functions for the monomials

In [ ]:
%run monomials

## Get the representers from the FE code

In [ ]:
%run femshape

space = Space(order=4, meshsize=3,L=1)

def rep_fem(x,y):
    Uret = []
    Vret = []
    H1ret = []
    H2ret = []
    Mret = []
    dxret = []
    dyret = []
    for count in range(np.shape(x)[0]):
        curve = np.vstack((x[count,:],y[count,:])).T
        current = Current(space, curve)
#        inv1 = shapecalc.compute_invariants(closed=True)
        representer = Representer(current)
        #U, V, H1, H2, M, invx, invy = current.calcM(ret_inv=True)
        U, V, H1, H2, M, invx, invy = (representer.H1x.vector()[:],
                                       representer.H1y.vector()[:],
                                       representer.H1,
                                       representer.H2,
                                       representer.M.array(),
                                       current.invariants[:,0],
                                       current.invariants[:,1],
                                      )
        Uret.append(U)
        Vret.append(V)
        H1ret.append(H1)
        H2ret.append(H2)
        Mret.append(M)
        dxret.append(invx)
        dyret.append(invy)
        #U[:,count], V[:,count], H1[count], H2[count] = shapecalc.calcM()
    return np.squeeze(np.array(Uret)), np.squeeze(np.array(Vret)), np.array(H1ret), np.array(H2ret), np.array(Mret), np.array(dxret), np.array(dyret)

In [ ]:
%run utils

In [ ]:
#Test 1: Construct 5 random variations of the circle and a figure of 8
npoints = 50
ncurves = 3
x = np.zeros((ncurves,npoints))
y = np.zeros((ncurves,npoints))

x[1:,:],y[1:,:] = randshapes(ncurves-1,npoints)

x[0], y[0] = figure_of_eight(npoints)

# Using the finite elements
U,V,H1,H2, M, dx, dy = rep_fem(x,y)
pe = np.transpose(np.array((U,V)),(1,0,2))
#pl.plot(U[2,:],V[2,:],'.')
ninv = np.shape(pe)[2]
pe = np.reshape(pe,(ncurves,ninv*2),order='F')
compute_pca(pe, x, y)

# Using the monomials
cl = (npoints-1)*np.ones((1,ncurves),dtype=int)
u = representer(x,y,ncurves,cl,npoints,10)
u1 = np.zeros((ncurves,np.shape(u)[1]*np.shape(u)[2]*2))
for i in range(ncurves):
    u1[i,:] = np.matrix.flatten(u[i,:,:,:])
compute_pca(u1, x, y)

In [ ]:
# Save a load of data into matlab file for easy comparison
import scipy.io as spi
ncurves = 200
npoints = 500
x,y = randshapes(ncurves,npoints)
spi.savemat('currents_data_200_500.mat',{'x':x,'y':y})

In [ ]:


# PCA on the representer space, not Euclidean space
ncurves = 32
data = spi.loadmat('currents_data_200_500.mat')
x = data['x']
y = data['y']
x = x[:ncurves,:]
y = y[:ncurves,:]
U,V,H1,H2,M,dx,dy = rep_fem(x,y)
pe = np.transpose(np.array((U,V)),(1,0,2))
#pl.plot(U[2,:],V[2,:],'.')
ninv = np.shape(pe)[2]
pe = np.reshape(pe,(ncurves,ninv*2),order='F')
compute_pca(pe, x, y)

cl = (npoints-1)*np.ones((1,ncurves),dtype=int)
u = representer(x,y,ncurves,cl,npoints,10)
u1 = np.zeros((ncurves,np.shape(u)[1]*np.shape(u)[2]*2))
for i in range(ncurves):
    u1[i,:] = np.matrix.flatten(u[i,:,:,:])
compute_pca(u1, x, y)

In [ ]:
# This version is like Robert's, does PCA on the currents of the curve (not the representer)
def current_pca(ncurves=32,npoints=500,N=10):
    pl.ion()
    cl = (npoints-1)*np.ones((1,ncurves),dtype=int)
    #x,y = randshapes(ncurves,npoints)
    data = spi.loadmat('currents_data_200_500.mat')
    x = data['x']
    y = data['y']
    x = x[:ncurves,:]
    y = y[:ncurves,:]
    #x[0,:] = 0.5*np.cos(np.linspace(0,2*np.pi,npoints,endpoint=False))
    #y[0,:] = 0.5*np.sin(2*np.linspace(0,2*np.pi,npoints,endpoint=False))
    x,y,curr,curr2 = getc(x,y,ncurves,cl,N)

    [Gi,G] = dualnorm(N)
    L = np.linalg.cholesky(G)
    Linv = np.linalg.inv(L)
    from scipy.linalg import block_diag
    #Gihalf = block_diag(Linv,Linv)
    Gihalf = block_diag(Linv)
    
    #Ghalf = np.linalg.cholesky(G).T

    # representer in Euclidean basis
    #ue = np.dot(Ghalf,u1)
    #print (np.sqrt(np.dot(u1.T,np.dot(G,u1))),np.linalg.norm(ue))
    #Gihalf = np.linalg.cholesky(Gi).T

    pe = np.zeros(np.shape(curr2))
    for i in range(ncurves):
        pe[i,:] = np.dot(Gihalf,curr2[i,:].T).T

    pca_opt(pe,x,y,'monomial',20)
    

    #curr2 = np.hstack((invx,invy))


In [ ]:
current_pca()

In [ ]:
# This version is like Robert's, does PCA on the currents of the curve (not the representer)
def fem_current_pca(ncurves=32,npoints=500,order=1,meshsize=10):
    pl.ion() 
    #x,y = randshapes(ncurves,npoints)
    data = spi.loadmat('currents_data_200_500.mat')
    x = data['x']
    y = data['y']
    x = x[:ncurves,:]
    y = y[:ncurves,:]
    #x[0,:] = 0.5*np.cos(np.linspace(0,2*np.pi,npoints,endpoint=False))
    #y[0,:] = 0.5*np.sin(2*np.linspace(0,2*np.pi,npoints,endpoint=False))
    
    U,V,H1,H2, G, invx, invy = rep_fem(x,y)
    G = np.squeeze(G[0,:,:])
    
    # For the H^1 metric use L = np.linalg.cholesky(G)
    L = np.linalg.cholesky(G)
    # For the H^2 metric use L = G
    #L=G
    
    Linv = np.linalg.inv(L)
    from scipy.linalg import block_diag
    Gihalf = block_diag(Linv,Linv)
    curr2 = np.hstack((invx,invy))

    # representer in Euclidean basis
    pe = np.zeros(np.shape(curr2))
    for i in range(ncurves):
        pe[i,:] = np.dot(Gihalf,curr2[i,:].T).T

    name = 'fem_'+str(order)+'_'+str(meshsize)
    pca_opt(pe,x,y,name,8)

In [ ]:
#space = FEMShapeInvariant(order=1, meshsize=10,L=1)
fem_current_pca(order=1,meshsize=10)
#shapecalc = fem.FEMShapeInvariant(order=1, meshsize=20,L=1)
#pe2, d = fem_current_pca()
#shapecalc = fem.FEMShapeInvariant(order=4, meshsize=20,L=1)
#pe = fem_current_pca()

d(0,1), d(0,2), d(0,3)
order 1, mesh = 10, 20, 40
(2.2949783713748606, 2.4006275209345507, 2.2316118057013319)
(1.6047046605854045, 1.6207933275335604, 1.6825498540372308)
(1.2664142928892494, 1.2783837142622918, 1.3259963291001282)

order 4, mesh = 5, 10, 20
(2.2997301392640863, 2.2460204102017722, 2.2746276352501957)
(1.7498901419675774, 1.7855146077354773, 1.7592703776709591)

d(0,1), d(0,2), d(1,2)
order 4, mesh = 5, 10, 20
(2.7899967862469111, 2.8270892665201925, 0.8246409669811765)
(2.8852668644254802, 2.9246305575121454, 1.0339455136119406)
(2.9389698376057898, 2.9713066427066308, 1.1315582359980916)